In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,TensorDataset
from gensim.models import Word2Vec
from models import RNNcVAE, GRUcVAE, LSTMcVAE, CNNClassifier
from config_dataset import custom_dataset
from accuracy import style_accuracy
from training_function import training, train_CNN

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sequence_length = 30
batch_size = 128
embedding_dim = 300
hidden_dim = 128
latent_dim = 100

In [4]:
train_loader, val_loader, embedding_dim, embedding_matrix, word2vec, idx2word, word2idx, vocab_size = custom_dataset('text_corpus/divina_commedia.txt',
                                                                                                                     'text_corpus/uno_nessuno_e_i_malavoglia.txt',
                                                                                                                     'text_corpus/lo_cunto_de_li_cunti.txt',
                                                                                                                    sequence_length,
                                                                                                                    embedding_dim,
                                                                                                                    batch_size,
                                                                                                                    0.9)

print('total number of training samples: ', len(train_loader.dataset))
print('total number of validation samples: ', len(val_loader.dataset))
print('vocab size: ', vocab_size)

total number of training samples:  45550
total number of validation samples:  5062
vocab size:  26607


In [5]:
sos_token = torch.full((1,),word2idx['<sos>'])
sos_token = sos_token.type(torch.LongTensor)

## Independent CNN Classifier

In [6]:
CNN_classif = CNNClassifier(embedding_matrix, 3, 2, [3,3])

style_params = sum(p.numel() for p in CNN_classif.parameters() if p.requires_grad)
print('Total parameters: ', style_params)

Total parameters:  3619


In [7]:
CNN_classif.load_state_dict(torch.load('pretrained/cnn_classifier.pth'))

<ipython-input-7-94d02519e97c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  CNN_classif.load_state_dict(torch.load('pretrained/cnn_classifier.pth'))


<All keys matched successfully>

In [8]:
wrong = 0.0
wrongy = []
with torch.no_grad():
    for i, (data, labels) in enumerate(val_loader):
        pred_style = torch.argmax(CNN_classif(data), dim=-1)
        label = torch.argmax(labels,dim=-1)
        wrong += torch.count_nonzero(label - pred_style)
        
print('Accuracy :', 1 - (wrong/(len(val_loader)*batch_size)).item(), '%')

Accuracy : 0.982812499627471 %


# RNN cVAE

In [9]:
rnn_cvae = RNNcVAE(embedding_matrix, idx2word, 3, hidden_dim, latent_dim, 2, sos_token, vocab_size)
rnn_cvae.number_parameters()

Total number of model parameters:  3648159


In [10]:
rnn_cvae.load_state_dict(torch.load('pretrained/rnn_cvae.pth'))

<ipython-input-10-859835cd591b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rnn_cvae.load_state_dict(torch.load('pretrained/rnn_cvae.pth'))


<All keys matched successfully>

### Change style
* Dante
* Italian
* Neapolitan

In [51]:
style = 'Dante'

In [53]:
style_label = {'Dante' : torch.FloatTensor([1,0,0]), 'Italian' : torch.FloatTensor([0,1,0]), 'Neapolitan' : torch.FloatTensor([0,0,1])}

label = style_label[style]
sentence, perplexity = rnn_cvae.sample(label, 40, 0.8)
sentence

'perch io a per ch io non ne tu ti nol conviene è la ma sì non si fu da uno a per che l come se si le l antica de da li la ma sì la gente nè'

In [ ]:
style_accuracy(rnn_cvae, CNN_classif, word2idx, name = 'RNN')

# GRU cVAE

In [11]:
gru_cvae = GRUcVAE(embedding_matrix, idx2word, 3, hidden_dim, latent_dim, 2, sos_token, vocab_size)
gru_cvae.number_parameters()

Total number of model parameters:  4000415


In [12]:
gru_cvae.load_state_dict(torch.load('pretrained/gru_cvae.pth'))

<ipython-input-12-1260851c4d4d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gru_cvae.load_state_dict(torch.load('pretrained/gru_cvae.pth'))


<All keys matched successfully>

### Change style
* Dante
* Italian
* Neapolitan

In [105]:
style = 'Dante'

In [106]:
style_label = {'Dante' : torch.FloatTensor([1,0,0]), 'Italian' : torch.FloatTensor([0,1,0]), 'Neapolitan' : torch.FloatTensor([0,0,1])}

label = style_label[style]
sentence, perplexity = gru_cvae.sample(label, 40, 0.9)
sentence

'fiso e dentro ch a ogne ma con sola nè quel i ma e per nè per e in sparì l e come l e per nel e e per sentiero o e su per che caro per è suo'

In [77]:
style_accuracy(gru_cvae, CNN_classif, word2idx, name = 'GRU')

TypeError: style_accuracy() missing 1 required positional argument: 'word2idx'

# LSTM cVAE

In [107]:
lstm_cvae = LSTMcVAE(embedding_matrix, idx2word, 3, hidden_dim, latent_dim, 2, sos_token, vocab_size)
lstm_cvae.number_parameters()

Total number of model parameters:  4088479


In [108]:
lstm_cvae.load_state_dict(torch.load('pretrained/lstm_cvae.pth'))

<ipython-input-108-0f6d58d4077f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lstm_cvae.load_state_dict(torch.load('pretrained/lstm_cvae.pth'))


<All keys matched successfully>

### Change style
* Dante
* Italian
* Neapolitan

In [127]:
style = 'Neapolitan'

In [130]:
style_label = {'Dante' : torch.FloatTensor([1,0,0]), 'Italian' : torch.FloatTensor([0,1,0]), 'Neapolitan' : torch.FloatTensor([0,0,1])}

label = style_label[style]
sentence, perplexity = lstm_cvae.sample(label, 40, 0.7)
sentence

'de lo munno e da lo re se ne scette sto piezzo e arrivato a lo quale cosa vedenno lo serpe pensiero e che non vole lo core ch è juta a la cantina e a tiempo venne lo dubbio'

In [ ]:
style_accuracy(lstm_cvae, CNN_classif, word2idx, name = 'LSTM')